# 作业3：构建一个NDArray库

在本次作业中，你将构建一个用于大多数深度学习系统底层处理的简单支持库：n维数组（又称NDArray）。到目前为止，你在很大程度上一直在使用numpy来实现这一目的，但本作业将引导你开发一个属于自己的（尽管功能有限得多）numpy变体，它将支持CPU和GPU两种后端。此外，与numpy（甚至像PyTorch这样的变体）不同，你不会简单地调用现有的高度优化的矩阵乘法或其他操作代码，而是要实际编写自己的版本，且其性能要与这些标准库背后的高度优化代码相当（从某种程度上来说，即“仅慢2-3倍”……这比 naive 代码要好得多，naive 代码很容易慢100倍以上）。这个类最终将集成到`needle`中，但在本次作业中，你只能专注于ndarray模块，因为这将是测试的唯一内容。

**注意**：为了避免耗尽Colab中有限的GPU资源，首先使用CPU运行时来编写代码和测试非GPU函数。在测试CUDA或GPU加速代码时，再切换到GPU运行时。这种方法可以确保高效利用GPU资源，并防止在关键任务期间耗尽资源。

In [3]:
import os

# 查看当前PATH环境变量
print("修改前的PATH:", os.environ.get("PATH", ""))

# 获取Anaconda的Python路径（你的Anaconda路径）
anaconda_path = "C:\\Users\\HSRASUS\\anaconda3"
anaconda_scripts = "C:\\Users\\HSRASUS\\anaconda3\\Scripts"

# 将Anaconda路径添加到PATH的最前面（优先级最高）
os.environ["PATH"] = f"{anaconda_path};{anaconda_scripts};{os.environ['PATH']}"

# 验证修改后的PATH
print("修改后的PATH:", os.environ.get("PATH", ""))

修改前的PATH: C:\Users\HSRASUS\anaconda3;C:\Users\HSRASUS\anaconda3\Scripts;C:\Users\HSRASUS\anaconda3;C:\Users\HSRASUS\anaconda3\Library\mingw-w64\bin;C:\Users\HSRASUS\anaconda3\Library\usr\bin;C:\Users\HSRASUS\anaconda3\Library\bin;C:\Users\HSRASUS\anaconda3\Scripts;C:\Users\HSRASUS\anaconda3\bin;C:\Users\HSRASUS\anaconda3\condabin;C:\Program Files\NVIDIA GPU Computing Toolkit\CUDA\v12.3\bin;C:\Program Files\NVIDIA GPU Computing Toolkit\CUDA\v12.3\libnvvp;D:\Vm\bin;C:\Program Files (x86)\Razer Chroma SDK\bin;C:\Program Files\Razer Chroma SDK\bin;C:\Program Files (x86)\Razer\ChromaBroadcast\bin;C:\Program Files\Razer\ChromaBroadcast\bin;C:\Program Files (x86)\Common Files\Intel\Shared Libraries\redist\intel64\compiler;C:\Windows\system32;C:\Windows;C:\Windows\System32\Wbem;C:\Windows\System32\WindowsPowerShell\v1.0;C:\Windows\System32\OpenSSH;C:\Program Files (x86)\NVIDIA Corporation\PhysX\Common;C:\Program Files\NVIDIA Corporation\NVIDIA NvDLISR;C:\Program Files\dotnet;E:\EasyShare\x86;E

In [ ]:
# Code to set up the assignment
from google.colab import drive
drive.mount('/content/drive')
%cd /content/drive/MyDrive/
!mkdir -p 10714
%cd /content/drive/MyDrive/10714
!git clone https://github.com/dlsys10714/hw3.git
%cd /content/drive/MyDrive/10714/hw3

!pip3 install --upgrade --no-deps git+https://github.com/dlsys10714/mugrade.git
!pip3 install pybind11

In [6]:
!make

Makefile:7: recipe for target 'lib' failed


make: mkdir: Command not found
make: *** [lib] Error 127


# make 命令

`make`命令会读取当前目录下的`Makefile`文件。`Makefile`包含了用于定义如何构建目标（如可执行文件或库）的规则。

对于`Makefile`中指定的每个目标，`make`会检查目标文件及其依赖项（如`.c`、`.cpp`或`.h`文件）的时间戳。如果任何依赖项最近被修改过，则必须重新构建该目标。

In [4]:
%set_env PYTHONPATH ./python
%set_env NEEDLE_BACKEND nd

env: PYTHONPATH=./python
env: NEEDLE_BACKEND=nd


In [4]:
import sys
sys.path.append('./python')

### 熟悉 NDArray 类

开始本作业时，你首先需要熟悉我们提供的 NDArray.py 类，它是本作业的起点。这段代码相当简洁（约 500 行，但其中很多是你将要实现的函数的注释）。

NDArray 类的核心是一个 Python 包装器，用于处理通用 n 维数组的运算。回想一下，几乎任何此类数组在内部都会存储为浮点值向量，即：
float data[size];
而对数组不同维度的实际访问则由其他字段（如数组形状、步长等）处理，这些字段指示了这个“扁平”数组如何映射到 n 维结构。为了达到合理的速度，“原始”运算（如加法、二元运算，以及更结构化的运算如矩阵乘法等）都需要在某种程度上用 C++ 等原生语言编写（包括调用 CUDA 等）。但很多运算，如转置、广播、矩阵子集化等，都可以通过调整数组的高级结构（如步长）来处理。

NDArray 类的设计理念是，所有处理数组结构的逻辑都用 Python 编写。只有真正执行扁平向量底层原始运算的低级代码（以及管理这些扁平向量的代码，因为它们可能需要在 GPU 上分配等）才用 C++ 编写。这种分离的具体性质可能在你完成作业的过程中会逐渐清晰，但一般来说，所有能在 Python 中完成的工作都在 Python 中完成；通常，例如，为了使底层 C++ 实现更简单，我们会大量调用.compact()（它会复制内存），这可能会带来一些效率损失。

更详细地说，NDArray 类中有五个字段需要你熟悉（注意，所有这些字段的实际类成员都带有下划线前缀，例如_handle、_strides 等，其中一些会作为公共属性暴露……在你的所有代码中，使用带下划线的内部版本是没问题的）。

- device：BackendDevice 类型的对象，它是一个简单的包装器，包含指向底层设备后端（如 CPU 或 CUDA）的链接。
- handle：存储数组底层内存的类对象。它被分配为 device.Array() 类型的类，不过这种分配都在提供的代码中（特别是 NDArray.make 函数）完成，你无需担心自己调用它。
- shape：一个元组，指定数组每个维度的大小。
- strides：一个元组，指定数组每个维度的步长。
- offset：一个整数，指示数组在底层 device.Array 内存中的实际起始位置（存储这个值很方便，这样我们可以更容易地管理指向现有内存的指针，而无需跟踪分配情况）。

通过操作这些字段，即使是纯 Python 代码也能执行许多所需的数组运算，如维度置换（即转置）、广播等。然后，对于原始数组操作调用，device 类有许多方法（用 C++ 实现）包含必要的实现。

有几点需要注意：

在内部，该类可以使用任何高效的方式对数据数组进行操作作为“设备”后端。例如，即使是 numpy 数组，我们也不是实际使用 numpy.ndarray 来表示 n 维数组，而是仅在 numpy 中表示一个“扁平”的 1D 数组，然后调用相关的 numpy 方法来实现所有需要的运算符对这个原始内存进行操作。这正是我们在 ndarray_backend_numpy.py 文件中所做的，它本质上提供了一个“存根引用”，所有操作都使用 numpy 完成。你可以使用这个类来帮助你更好地调试自己为“原生”CPU 和 GPU 后端编写的“真实”实现。

对于你的许多 Python 实现来说，NDArray.make 调用尤为重要：
def make(shape, strides=None, device=None, handle=None, offset=0):
该函数创建一个具有给定形状、步长、设备、句柄和偏移量的新 NDArray。如果未指定句柄（即不引用预先存在的内存），则该调用将分配所需的内存，但如果指定了句柄，则不会分配新内存，而是新的 NDArray 指向与旧的相同的内存。让尽可能多的函数不分配新内存对高效实现很重要，因此你会希望在很多情况下使用这个调用来实现这一点。

NDArray 有一个.numpy()调用，可以将数组转换为 numpy 数组。这与“numpy_device”后端不同：这会创建一个与给定 NDArray 等效的实际 numpy.ndarray，即相同的维度、形状等，尽管步长不一定相同（Pybind11 在返回这种矩阵时会重新分配内存，这可能会改变步长）。

### 第一部分：Python 数组操作

作为类的起点，请在 `ndarray.py` 文件中实现以下函数：

- `reshape()`
- `permute()`
- `broadcast_to()`
- `__getitem__()`

这些函数的输入/输出都在函数存根的文档字符串中进行了描述。需要重点强调的是，**这些函数都不应重新分配内存**，而应返回与 `self` 共享相同内存的 NDArray，只需通过巧妙地操作形状/步长等参数来实现必要的转换。

开始时，你可以参考课堂幻灯片中关于转置、广播和切片运算符的提示。

需要注意的一点是，与 numpy 不同，`__getitem__()` 调用永远不会改变数组的维度数量。例如，对于一个 2D NDArray `A`，`A[2,2]` 将返回一个仍为 2D 的数组，其中包含一行一列。再例如，`A[:4,2]` 将返回一个具有 4 行 1 列的 2D NDArray。

在本节中，你可以依赖 `ndarray_backend_numpy.py` 模块来实现所有代码。你也可以查看等效的 numpy 操作的结果（测试用例应该会说明这些操作是什么）。

实现这些函数后，你应该通过/提交以下测试。请注意，我们在下面的测试中测试了所有这四个函数，你可以在实现每个额外的函数时逐步尝试通过更多的测试。

In [5]:
!python -m pytest -v -k "(permute or reshape or broadcast or getitem) and cpu and not compact"

============================= test session starts =============================
platform win32 -- Python 3.12.3, pytest-7.4.4, pluggy-1.0.0 -- C:\Users\HSRASUS\anaconda3\python.exe
cachedir: .pytest_cache
rootdir: D:\Desktop\大三上学期学习\模型并行\ML_system\hw3
plugins: anyio-4.2.0
collecting ... collected 0 items / 1 error

=================================== ERRORS ====================================
_________________ ERROR collecting tests/hw3/test_ndarray.py __________________
ImportError while importing test module 'D:\Desktop\大三上学期学习\模型并行\ML_system\hw3\tests\hw3\test_ndarray.py'.
Hint: make sure your test modules/packages have valid Python names.
Traceback:
C:\Users\HSRASUS\anaconda3\Lib\importlib\__init__.py:90: in import_module
    return _bootstrap._gcd_import(name[level:], package, level)
tests\hw3\test_ndarray.py:4: in <module>
    import needle as ndl
E   ModuleNotFoundError: No module named 'needle'
=========================== short test summary info ===========================
ERR

In [ ]:
!python3 -m mugrade submit "YOUR KEY HERE" -k "ndarray_python_ops"

### 第二部分：CPU 后端 - Compact 和 setitem

在 `ndarray_backend_cpu.cc` 中实现以下函数：
* `Compact()`
* `EwiseSetitem()`
* `ScalarSetitem()`

为了理解为什么这些函数被归为同一类，我们来分析 `Compact()` 函数的实现。回想一下，一个矩阵被认为是**紧凑的（compact）**，如果它以"行优先（row-major）"形式连续存储在内存中（实际上是更高维数组的行优先推广），即最后一个维度优先， followed by the second to last dimension, etc, all the way to the first. 在我们的实现中，我们还要求分配的后端数组总大小等于该数组的大小（即，底层数组在数组数据前后不能有任何额外数据，这意味着 `.offset` 字段必须为零）。

现在让我们以一个 3D 数组为例，看看 compact 操作是如何工作的。这里的 `shape` 和 `strides` 是要被压缩的矩阵（即压缩前）的形状和步长。

```c++
cnt = 0;
for (size_t i = 0; i < shape[0]; i++)
    for (size_t j = 0; j < shape[1]; j++)
        for (size_t k = 0; k < shape[2]; k++)
            out[cnt++] = in[strides[0]*i + strides[1]*j + strides[2]*k];
```

换句话说，我们正在从矩阵的基于步长的表示转换为纯粹的连续表示。

实现 `Compact()` 的挑战在于，你希望该方法能适用于任何输入维度。为不同的固定维度大小的数组编写专用实现很容易，但对于通用实现，你需要思考如何执行相同的操作，即有效地实现"可变数量的 for 循环"。提示一下，一种方法是维护一个索引向量（大小等于维度数），然后在循环中手动递增它们（包括当任何一个索引达到其最大大小时的"进位"操作）。

不过，如果你在这里确实遇到困难，你可以利用一个事实：我们可能不会要求你处理超过 6 维的矩阵（尽管我们会使用 6 维，用于我们在课堂上讨论的 im2col 操作）。


#### 与 setitem 的联系
setitem 功能虽然看起来很不一样，但实际上与 `Compact()` 密切相关。`__setitem()__` 是在设置对象的某些元素时调用的 Python 函数，例如：
```python
A[::2,4:5,9] = 0 # 或者 = some_other_array
```

你会如何实现这个功能呢？在上面的 `__getitem()__` 调用中，你已经实现了一种无需复制即可获取矩阵子集的方法（只需修改步长）。但是你实际上如何**设置**这个数组的元素呢？在本作业的几乎所有其他设置中，我们在设置输出数组的项之前会调用 `.compact()`，但在这种情况下这行不通：调用 `.compact()` 会将子集数组复制到一些新内存，但 `__setitem__()` 调用的全部意义在于我们想要修改现有内存。

如果你思考一下，你会意识到答案看起来很像 `.compact()` 的反向操作。如果我们想要将一个（本身已经紧凑的）右值矩阵分配给 `__getitem()__` 的结果，那么我们需要像这样使用输出矩阵的 `shape` 和 `stride`。然后我们可以如下实现 setitem 调用：

```c++
cnt = 0;
for (size_t i = 0; i < shape[0]; i++)
    for (size_t j = 0; j < shape[1]; j++)
        for (size_t k = 0; k < shape[2]; k++)
            out[strides[0]*i + strides[1]*j + strides[2]*k] = in[cnt++]; // 或者 "= val;"
```

由于这种相似性，如果你以模块化的方式实现你的索引策略，你将能够在 `Compact()` 调用与 `EwiseSetitem()` 和 `ScalarSetitem()` 调用之间重用它。

**注意**：在执行测试之前不要忘记运行 make 来重新构建你修改过的 C++ 代码。

In [ ]:
!python3 -m pytest -v -k "(compact or setitem) and cpu"

In [ ]:
!python3 -m mugrade submit "YOUR KEY HERE" -k "ndarray_cpu_compact_setitem"

## Part 3: CPU Backend - Elementwise and scalar operations

Implement the following functions in `ndarray_backend_cpu.cc`:

* `EwiseMul()`, `ScalarMul()`
* `EwiseDiv()`, `ScalarDiv()`
* `ScalarPower()`
* `EwiseMaximum()`, `ScalarMaximum()`
* `EwiseEq()`, `ScalarEq()`
* `EwiseGe()`, `ScalarGe()`
* `EwiseLog()`
* `EwiseExp()`
* `EwiseTanh()`

You can look at the included
`EwiseAdd()` and `ScalarAdd()` functions (plus the invocations from `NDArray` in order to understand the required format of these functions.

Note that unlike the remaining functions mentioned here, we do not include function stubs for each of these functions.  This is because, while you can implement these naively just through implementing each function separately, though this will end up with a lot of duplicated code.  You're welcome to use e.g., C++ templates or macros to address this problem (but these would only be exposed internally, not to the external interface).

In [ ]:
!python3 -m pytest -v -k "(ewise_fn or ewise_max or log or exp or tanh or (scalar and not setitem)) and cpu"

In [ ]:
!python3 -m mugrade submit "YOUR KEY HERE" -k "ndarray_cpu_ops"

## Part 4: CPU Backend - Reductions


Implement the following functions in `ndarray_backend_cpu.cc`:

* `ReduceMax()`
* `ReduceSum()`

In general, the reduction functions `.max()` and `.sum()` in NDArray take the max or sum across a specified axis specified by the `axis` argument (or across the entire array when `axis=None`); note that we don't support axis being a set of axes, though this wouldn't be too hard to add if you desired (but it's not in the interface you should implement for the homework).

Because summing over individual axes can be a bit tricky, even for compact arrays, these functions (in Python) in Python simplify things by permuting the last axis to the be the one reduced over (this is what the `reduce_view_out()` function in NDArray does), then compacting the array.  So for your `ReduceMax()` and `ReduceSum()` functions you implement in C++, you can assume that both the input and output arrays are contiguous in memory, and you want to just reduce over contiguous elements of size `reduce_size` as passed to the C++ functions. 

In [ ]:
!python3 -m pytest -v -k "reduce and cpu"

In [ ]:
!python3 -m mugrade submit "YOUR KEY HERE" -k "ndarray_cpu_reductions"

## Part 5: CPU Backend - Matrix multiplication

Implement the following functions in `ndarray_backend_cpu.cc`:

* `Matmul()`
* `MatmulTiled()`
* `AlignedDot()`

The first implementation, `Matmul()` can use the naive three-nested-for-loops algorithm for matrix multiplication.  However, the `MatmulTiled()` performs the same matrix multiplication on memory laid out in tiled form, i.e., as a contiguous 4D array
```c++
float[M/TILE][N/TILE][TILE][TILE];
```

Make sure to initialize the output matrix to zero before populating it with the correct values during matrix multiplication.

Note that the Python `__matmul__` code already does the conversion to tiled form when all sizes of the matrix multiplication are divisible by `TILE`, so your code just needs to implement the multiplication in this form.  In order to make the methods efficient, you will want to make use of (after you implement it), the `AlignedDot()` function, which will enable the compiler to efficiently make use of vector operations and proper caching.  The output matrix will also be in the tiled form above, and the Python backend will take care of the conversion to a normal 2D array.

Note that in order to get the most speedup possible from you tiled version, you may want to use the clang compiler with colab instead of gcc.  To do this, run the following command before building your code.

In [ ]:
!python3 -m pytest -v -k "matmul and cpu"

In [ ]:
!python3 -m mugrade submit "YOUR KEY HERE" -k "ndarray_cpu_matmul"

In [ ]:
!export CXX=/usr/bin/clang++ && make

## Part 6: CUDA Backend - Compact and setitem

Implement the following functions in `ndarray_backend_cuda.cu`:
* `Compact()`
* `EwiseSetitem()`
* `ScalarSetitem()`


For this portion, you'll implement the compact and setitem calls in the CUDA backend.  This is fairly similar to the C++ version, however, depending on how you implemented that function, there could also be some substantial differences.  We specifically want to highlight a few differences between the C++ and the CUDA implementations, however.

First, as with the example functions implemented in the CUDA backend code, for all the functions above you will actually want to implement two functions: the basic functions listed above that you will call from Python, and the corresponding CUDA kernels that will actually perform the computation.  For the most part, we only provide the prototype for the "base" function in the `ndarray_backend_cuda.cu` file, and you will need to define and implement the kernel function yourself.  However, to see how these work, for the `Compact()` call we are providing you with the _complete_ `Compact()` call, and the function prototype for the `CompactKernel()` call.

One thing you may notice is the seemingly odd use of a `CudaVec` struct, which is a struct used to pass shape/stride parameters.  In the C++ version we used the STL `std::vector` variables to store these inputs (and the same is done in the base `Compact()` call, but CUDA kernels cannot operation on STL vectors, so something else is needed).  Furthermore, although we _could_ convert the vectors to normal CUDA arrays, this would be rather cumbersome, as we would need to call `cudaMalloc()`, pass the parameters as integer pointers, then free them after the calls.  Of course such memory management is needed for the actual underlying data in the array, but it seems like overkill to do it for just passing a variable-sized small tuple of shape/stride values.  The solution is to create a struct that has a "maximize" size for the number of dimensions an array can have, and then just store the actual shape/stride data in the first entries of these fields.  This is all done by the included `CudaVec` struct and `VecToCuda()` function, and you can just use these as provided for all the CUDA kernels that require passing shape/strides to the kernel itself.

The other (more conceptual) big difference between the C++ and CUDA implementations of `Compact()` is that in C++ you will typically loop over the elements of the non-compact array sequentially, which allows you to perform some optimizations with respect to computing the corresponding indices between the compact and non-compact arrays.  In CUDA, you cannot do this, and will need to implement code that can directly map from an index in the compact array to one in the strided array.

As before, we recommend you implement your code in such as way that it can easily be re-used between the `Compact()`, and `Setitem()` calls.  As a short note, remember that if you want to call a (separate, non-kernel) function from kernel code, you need to define it as a `__device__` function. In CUDA, `__global__` is used to define a function that runs on the GPU and is called from the CPU host, while `__device__` defines a function that runs and is called only on the GPU from other GPU function code


In [ ]:
!python3 -m pytest -v -k "(compact or setitem) and cuda"

In [ ]:
!python3 -m mugrade submit "YOUR KEY HERE" -k "ndarray_cuda_compact_setitem"

## Part 7: CUDA Backend - Elementwise and scalar operations

Implement the following functions in `ndarray_backend_cuda.cu`:

* `EwiseMul()`, `ScalarMul()`
* `EwiseDiv()`, `ScalarDiv()`
* `ScalarPower()`
* `EwiseMaximum()`, `ScalarMaximum()`
* `EwiseEq()`, `ScalarEq()`
* `EwiseGe()`, `ScalarGe()`
* `EwiseLog()`
* `EwiseExp()`
* `EwiseTanh()`

Again, we don't provide these function prototypes, and you're welcome to use C++ templates or macros to make this implementation more compact.  You will also want to uncomment the appropriate regions of the Pybind11 code once you've implemented each function.

In [ ]:
!python3 -m pytest -v -k "(ewise_fn or ewise_max or log or exp or tanh or (scalar and not setitem)) and cuda"

In [ ]:
!python3 -m mugrade submit "YOUR KEY HERE" -k "ndarray_cuda_ops"

## Part 8: CUDA Backend - Reductions


Implement the following functions in `ndarray_backend_cuda.cu`:

* `ReduceMax()`
* `ReduceSum()`

You can take a fairly simplistic approach here, and just use a separate CUDA thread for each individual reduction item: i.e., if there is a 100 x 20 array you are reducing over the second dimension, you could have 100 threads, each of which individually processed its own 20-dimensional array..  This is particularly inefficient for the `.max(axis=None)` calls, but we won't worry about this for the time being.  If you want a more industrial-grade implementation, you use a hierarchical mechanism that first aggregated across some smaller span, then had a secondary function that aggregated across _these_ reduced arrays, etc.  But this is not needed to pass the tests.

In [ ]:
!python3 -m pytest -v -k "reduce and cuda"

In [ ]:
!python3 -m mugrade submit "YOUR KEY HERE" -k "ndarray_cuda_reductions"

## Part 9: CUDA Backend - Matrix multiplication

Implement the following functions in `ndarray_backend_cuda.cu`:

* `Matmul()`

Finally, as your final exercise, you'll implement matrix multiplication on the GPU.  Your implementation here can roughly follow the presentation in class.  While you can pass the tests using fairly naive code here (i.e., you could just have a separate thread for each (i,j) location in the matrix, doing the matrix multiplication efficiently (to make it actually faster than a CPU version) requires cooperative fetching and the block shared memory register tiling covered in class.  Try to implement using these methods, and see how much faster you can get your code than the C++ (or numpy) backends.


In [ ]:
!python3 -m pytest -v -k "matmul and cuda"

In [ ]:
!python3 -m mugrade submit "YOUR KEY HERE" -k "ndarray_cuda_matmul"